# Service Areas for Social Facilities

In [1]:
import geopandas as gpd
import osmnx as ox
import pandana as pdna
from geosnap import DataStore
from geosnap.analyze import isochrones_from_gdf
from geosnap.io import get_acs, get_network_from_gdf

In [2]:
%load_ext watermark
%watermark -a 'eli knaap' -v -d -u -p geopandas,geosnap
%load_ext autoreload
%autoreload 2

Author: eli knaap

Last updated: 2024-03-03

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.18.1

geopandas: 0.14.2
geosnap  : 0.12.1.dev9+g3a1cb0f6de61.d20240110



**Note this notebook requires osmnx**

One way of thinking about isochrones is considering them as service areas. That is, given some travel budget (in time, distance, transit fare, etc), the isochrone represents the service area accessible within that budget.

Imagine we were interested in the service areas around social facilities in Syracuse

In [3]:
datasets = DataStore()

In [4]:
tracts = get_acs(datasets, msa_fips="45060", years=[2018], level="tract")

In [6]:
facilities = ox.features.features_from_polygon(
    tracts.unary_union, {"amenity": "social_facility"}
)

In [7]:
facilities = facilities[facilities.geometry.type == "Point"]

In [8]:
tracts = tracts.to_crs(tracts.estimate_utm_crs())

In [9]:
facilities = facilities.to_crs(tracts.crs)

In [10]:
facilities.explore(style_kwds=dict(radius=4))

## Walking Service Areas

In [11]:
get_network_from_gdf?

Signature:
get_network_from_gdf(
    gdf,
    network_type='walk',
    twoway=False,
    add_travel_times=False,
    default_speeds=None,
)
Docstring:
Create a pandana.Network object from a geodataframe (via OSMnx graph).

Parameters
----------
gdf : geopandas.GeoDataFrame
    dataframe covering the study area of interest; this should be stored in lat/long
    (epsg 4326). Note the first step is to take the unary union of this dataframe,
    which is expensive, so large dataframes may be time-consuming
network_type : str, {"all_private", "all", "bike", "drive", "drive_service", "walk"}
    the type of network to collect from OSM (passed to `osmnx.graph_from_polygon`)
    by default "walk"
twoway : bool, optional
    Whether to treat the pandana.Network as directed or undirected. For a directed network,
    use `twoway=False` (which is the default). For an undirected network (e.g. a
    walk network) where travel can flow in both directions, the network is more
    efficient when twoway

In [12]:
walk_net = get_network_from_gdf(tracts)

/Users/knaaptime/Dropbox/projects/geosnap/geosnap/io/networkio.py:68: UserWarning: GeoDataFrame is stored in coordinate system EPSG:32618 so the pandana.Network will also be stored in this system
  warn(


Generating contraction hierarchies with 10 threads.
Setting CH node vector of size 86033
Setting CH edge vector of size 226066
Range graph removed 216738 edges of 452132
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


In [13]:
isochrones_from_gdf?

Signature:
isochrones_from_gdf(
    origins,
    threshold,
    network,
    network_crs=None,
    reindex=True,
    hull='shapely',
    ratio=0.2,
    allow_holes=False,
)
Docstring:
Create travel isochrones for several origins simultaneously

Parameters
----------
origins : geopandas.GeoDataFrame
    a geodataframe containing the locations of origin point features
threshold: float
    maximum travel distance to define the isochrone, measured in the same
    units as edges_df in the pandana.Network object. If the network was
    created with pandana this is usually meters; if it was created with
    urbanaccess this is usually travel time in minutes.
network : pandana.Network
    pandana Network instance for calculating the shortest path isochrone
    for each origin feature
network_crs : str, int, pyproj.CRS (optional)
    the coordinate system used to store x and y coordinates in the passed
    pandana network. If None, the network is assumed to be stored in the 
    same CRS as the

### Comparing Hull algorithms

To create the service area, we need to bound the set of reachable intersections using some kind of polygon. The resolution of the service area is dependent on the resolution of the network (i.e. since geosnap and pandana do not interpolate along the road network, greater intersection density will result in a more well-defined polygon).

There are different bounding-polygon algorithms to choose from. The default is shapely's [`concave_hull`](https://shapely.readthedocs.io/en/stable/reference/shapely.concave_hull.html) implementation, with the [alpha_shape_auto](https://pysal.org/libpysal/generated/libpysal.cg.alpha_shape_auto.html) algorithm from libpysal available as an alternative

In [14]:
alpha = isochrones_from_gdf(
    facilities, network=walk_net, threshold=2000, hull="libpysal"
)

The alpha shape version of the concave hull is the most resource intensive because it tries to optimize the alpha parameter. This also makes it the slowest.

In [15]:
m = alpha.explore()
facilities.explore(m=m, color="black", style_kwds=dict(radius=3))

In [16]:
ch01 = isochrones_from_gdf(
    facilities, network=walk_net, threshold=2000, hull="shapely", ratio=0.1
)

The concave hull algorithm in shapely does not do automatic optimization, but requires setting a `ratio` parameter, with smaller values resulting in tighter bounding polygons. If the ratio parameter is set too low, the bounding polygon can be *too* tight

In [17]:
m = ch01.explore()
facilities.explore(m=m, color="black", style_kwds=dict(radius=3))

In [18]:
ch02 = isochrones_from_gdf(
    facilities, network=walk_net, threshold=2000, hull="shapely", ratio=0.2
)

In [19]:
m = ch02.explore()
facilities.explore(m=m, color="black", style_kwds=dict(radius=3))

## Driving Service Areas

Be *very* careful with driving times... There are lots of edges with no speed information (requiring us to make strong assumptions), and even at best, these represent free-flow conditions.

To get an automobile network, change  `network_type='drive'` and  `add_travel_times=True`

In [20]:
drive_net = get_network_from_gdf(tracts, network_type="drive", add_travel_times=True)

/Users/knaaptime/Dropbox/projects/geosnap/geosnap/io/networkio.py:68: UserWarning: GeoDataFrame is stored in coordinate system EPSG:32618 so the pandana.Network will also be stored in this system
  warn(


Generating contraction hierarchies with 10 threads.
Setting CH node vector of size 24608
Setting CH edge vector of size 65928
Range graph removed 65088 edges of 131856
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


When using travel-time based impedance, travel times are [measured **in seconds**](https://osmnx.readthedocs.io/en/stable/internals-reference.html#osmnx-speed-module)

In [21]:
# 10 minute drive-shed

drive_chrone = isochrones_from_gdf(
    facilities, network=drive_net, threshold=600, ratio=0.2
)

In [22]:
# look at only the first record

m = drive_chrone.iloc[[1]].explore()
facilities.iloc[[1]].explore(m=m, color="black", style_kwds=dict(radius=6))

/Users/knaaptime/mambaforge/envs/geosnap/lib/python3.11/site-packages/pyproj/transformer.py:820: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return self._transformer._transform_point(


In [23]:
drive_net.edges_df

,from,to,travel_time
0,211905525,212550604,40.6
1,211905525,6291988938,166.9
2,212028019,212827209,55.3
3,212533433,212533484,4.4
4,212533433,212533444,12.6
...,...,...,...
65923,10917595296,213148992,19.6
65924,10942234832,213009542,24.0
65925,10942234832,213045439,4.3
65926,10942234832,213177620,34.8


Remember also this is directed travel. These isochrones represent the area reachable *from* each social service provider; they do not necesssarily represent the origins who can reach the provider within a 10 minute drive. That is, drive networks are directed (and usually asymmetric).